In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
import torch

In [4]:
df= pd.read_csv("/Users/sabbirahamedsaown/Desktop/Final Year Project/BERT_Dis/balanced_dataset.csv")

In [3]:
df

,text,label
0,Miss him yet?Here s an up-close look at a qual...,1
1,NEW YORK (Reuters) - Democratic presidential h...,0
2,,1
3,He is a billionaire who made his fortune in re...,0
4,***WARNING***This video will make your blood b...,1
...,...,...
1995,WASHINGTON (Reuters) - Longtime Trump communic...,0
1996,Will the Democrats ultimately choose a Marxist...,1
1997,"Windham, New Hampshire (CNN) Donald Trump pump...",0
1998,0 Add Comment \nCHECK it out! With 360 video n...,1


In [4]:
df.head(10)

,text,label
0,Miss him yet?Here s an up-close look at a qual...,1
1,NEW YORK (Reuters) - Democratic presidential h...,0
2,,1
3,He is a billionaire who made his fortune in re...,0
4,***WARNING***This video will make your blood b...,1
5,"MIAMI — Dalia Carmeli, who drives a trolley...",0
6,Mikhail Saakashvili quits as Odessa governor ...,1
7,GUATEMALA CITY (Reuters) - U.S. Homeland Secur...,0
8,"November 17, 2016 102 Google's chief executive...",1
9,GENEVA (Reuters) - A U.S.-funded famine survey...,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1999 non-null   object
 1   label   2000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 31.4+ KB


In [6]:
df.describe()

,label
count,2000.000000
mean,0.500000
std,0.500125
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


In [7]:
df.isnull().sum()

text     1
label    0
dtype: int64

In [5]:
df= df.fillna('')

In [9]:
df.isnull().sum()

text     0
label    0
dtype: int64

In [10]:
df.columns

Index(['text', 'label'], dtype='object')

In [6]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(df['text'], df['label'], test_size=0.2)

In [7]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Load the DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Load the DistilBERT model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Tokenize the texts
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=512)

In [9]:
import torch

class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_encodings, train_labels.tolist())
val_dataset = NewsDataset(val_encodings, val_labels.tolist())


In [10]:
#dataset initialization check
print(f"Train dataset: {train_dataset}")
print(f"Validation dataset: {val_dataset}")


Train dataset: <__main__.NewsDataset object at 0x7fb192c501c0>
Validation dataset: <__main__.NewsDataset object at 0x7fb192c50040>


In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader


# Training arguments with FP16 enabled
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    fp16=False,
     dataloader_num_workers=4)
    
class CustomTrainer(Trainer):
    def get_train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.args.train_batch_size,
            num_workers=self.args.dataloader_num_workers,
            pin_memory=True
        )

    def get_eval_dataloader(self, eval_dataset=None):
        eval_dataset = eval_dataset if eval_dataset is not None else self.eval_dataset
        return DataLoader(
            eval_dataset,
            batch_size=self.args.eval_batch_size,
            num_workers=self.args.dataloader_num_workers,
            pin_memory=True
        )

# Initialize the trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)


print(f"TrainingArguments: {training_args}")
print(f"Train dataset: {train_dataset}")
print(f"Validation dataset: {val_dataset}")

# Start training
trainer.train()



/Users/sabbirahamedsaown/opt/anaconda3/lib/python3.9/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


TrainingArguments: TrainingArguments(
_n_gpu=0,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=4,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/sabbirahamedsaown/opt/anaconda3/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/sabbirahamedsaown/opt/anaconda3/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'NewsDataset' on <module '__main__' (built-in)>


In [ ]:
trainer.evaluate()


In [ ]:
model.save_pretrained('path_to_save_model')
tokenizer.save_pretrained('path_to_save_tokenizer')


In [ ]:
model = DistilBertForSequenceClassification.from_pretrained('path_to_save_model')
tokenizer = DistilBertTokenizer.from_pretrained('path_to_save_tokenizer')


In [ ]:
def predict(text):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.softmax(logits, dim=-1)
    return predictions.numpy()

# Example prediction
text = "GENEVA (Reuters) - The United Nations called on Monday for a humanitarian pause in the Yemeni capital of Sanaa on Tuesday to allow civilians to leave their homes, aid workers to reach them, and the wounded to get medical care. Jamie McGoldrick, U.N. humanitarian coordinator in Yemen, said in a statement that the streets of Sanaa had become  battlegrounds  and that aid workers  remain in lockdown .  Thus, I call on all parties to the conflict to urgently enable a humanitarian pause on Tuesday 5 December, between 10:00 a.m. and 16:00 p.m. to allow civilians to leave their homes and seek assistance and protection and to facilitate the movement of aid workers to ensure the continuity of life-saving programs,  he said.  McGoldrick warned the warring parties that any deliberate attacks against civilians, and against civilian and medical infrastructure, are  clear violations of international humanitarian law and may constitute war crimes ."
prediction = predict(text)
print(prediction)


In [ ]:
def predict(text):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.softmax(logits, dim=-1)
   
    # This line picks 'true' if the confidence of class 0 is higher than class 1, else 'false'
    is_true = predictions[0, 1] > predictions[0, 0]
    return "Reliable" if is_true else "Not Reliable"

# Example prediction
text = "GENEVA (Reuters) - The United Nations called on Monday for a humanitarian pause in the Yemeni capital of Sanaa on Tuesday to allow civilians to leave their homes, aid workers to reach them, and the wounded to get medical care. Jamie McGoldrick, U.N. humanitarian coordinator in Yemen, said in a statement that the streets of Sanaa had become battlegrounds and that aid workers remain in lockdown. Thus, I call on all parties to the conflict to urgently enable a humanitarian pause on Tuesday 5 December, between 10:00 a.m. and 16:00 p.m. to allow civilians to leave their homes and seek assistance and protection and to facilitate the movement of aid workers to ensure the continuity of life-saving programs, he said. McGoldrick warned the warring parties that any deliberate attacks against civilians, and against civilian and medical infrastructure, are clear violations of international humanitarian law and may constitute war crimes."
prediction = predict(text)
print(prediction)


In [ ]:
def predict(text):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.softmax(logits, dim=-1)
  
    # This line picks 'true' if the confidence of class 0 is higher than class 1, else 'false'
    is_true = predictions[0, 1] > predictions[0, 0]
    return "Reliable" if is_true else "Not Reliable"

# Example prediction
text = "Before we get too far into this story, it needs to be pointed out that the 18 year old teen who was just sentenced by a judge for having a homemade Nazi tattoo, was only 15 years old at the time, and has since admitted he made a mistake. The Afghan  rapefugee  on the other hand, was 17 years old when he brutally raped a 72 year old woman. So basically, we re comparing a 15 year old who was utilizing his freedom of expression (whether we agree with him or not is irrelevant) with a violent 17 year old rapist.An 18-year-old Afghan  refugee  was sentenced to 20 months in prison for the anal rape of a 72-year-old woman. Now the Austrian justice system has once again shown us its quality. A 20-year-old, who was 15 at the time, the  offence  was committed, has just been sentenced to 19 months in prison for giving a home-made swastika tattoo.Three young people have been found guilty of engaging in Nazi activities after they drew Nazi tattoos and posted photos of them online.A 20-year-old, who was found to have a German war flag used by the Nazis and portraits of Nazi Socialists in his room, was sentenced by a youth court in Salzburg to 19 months in prison, three months unconditional.He and a 19-year-old accomplice were found guilty of using a pin and eyeliner to tattoo a hand-sized swastika onto someone s chest and then posting the photos online.In addition to the tattooing, the 20-year-old was also accused of shouting Nazi slogans out of his window and singing the song  Polaken-Tango  by the banned neo-Nazi rock group Landser in front of his brother and his brother s girlfriend.He was arrested after his brother called the police, who found him in his room wrapped in the war flag and surrounded by photos of prominent Nazis.He told the court that he regretted his actions, which took place in August 2011 when he was 15-years-old, and that he had now changed.Pleading guilty, he said:  I now know that this is nonsense.  He was charged separately for his Facebook account, where he also posted pictures of himself making Nazi salutes, which he told judge Bettina Maxones-Kurkowski he had done to  show others that he belongs here . Via: The LocalOn Wednesday at the State Court in Vienna s Neustadt an 18-year-old Afghan was sentenced to 20 months in prison without parole for rape. He also has to pay 5000 euros compensation to the 72-year-old woman, who has been very marked by the attack.The incident on 1 September last year created a sensation: the penioner was walking her old dog in the Schwechat meadow near Traiskirchen in Lower Austria, when she encountered two young asylum seekers swimming in the river. According to an acquaintance of the victim in the witness stand, the boys were  also very nice  at first, they even helped the woman over an embankment.  But then one them fell upon her from behind. However, DNA traces confirmed that the pensioner was anally raped. The then 17-year-old Afghan was quickly caught. He does not dispute the crime, but says he was drunk. His friend says he wasn t aware of the rape.Her daughter Sylvia   a committed refugee helper   believes and hopes that her mother was unconscious during the crime. What does Christine F. feel for her tormentor? Despite all of the torment, she even feels some sympathy."
prediction = predict(text)
print(prediction)
